<a href="https://colab.research.google.com/github/MPrazzoli/corpsol/blob/main/Script_buono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install QuantLib

     |████████████████████████████████| 20.1MB 1.8MB/s 


In [4]:
import os
import math
import numpy as np
import pandas as pd
from QuantLib import *
from datetime import date

**Inserimento degli Input**

In [5]:
try:
  underlying = input('Inserire il sottostante: ')
  if underlying == '':
    underlying = 'EUR3M'
except:
  underlying = 'EUR3M'

try:
  amortization_type = input('Inserire il piano di ammortamento: ')
  if amortization_type == '':
    amortization_type = 'Bullet'
except:
  amortization_type = 'Bullet'

try:
  notional = float(input('Inserire il nozionale: '))
except:
  notional = 100000.0

try:
  amortization_rate = float(input('Inserire il tasso annuo: '))
except:
  amortization_rate = .05

try:
  maturity_label = input('Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): ')
  if maturity_label == '':
     maturity_label = '5Y'
except:
  maturity_label = '5Y'

try:
  ref_date = Date(input('Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: '),'%d/%m/%Y')
  if ref_date == '':
    ref_date = Date(date.today().day,date.today().month,date.today().year)
except:
  ref_date = Date(date.today().day,date.today().month,date.today().year)

try:
  tenor = int(input('Inserire il periodo che intercorre tra le rate in numero di mesi: '))
except:
  tenor = 3

try:
  business_convention = input('Inserire la convenzione:')
  if business_convention == '' or business_convention == 'ModifiedFollowing':
    business_convention = ModifiedFollowing
except:
  business_convention = ModifiedFollowing

try:
  date_gen_rule = input('Inserire la regola per il calcolo delle date:')
  if date_gen_rule == '' or date_gen_rule == 'Forward':
    date_gen_rule = DateGeneration.Forward
  else: 
    date_gen_rule = DateGeneration.Backward
except:
  date_gen_rule = DateGeneration.Forward

Inserire il sottostante: 
Inserire il piano di ammortamento: 
Inserire il nozionale: 
Inserire il tasso annuo: 
Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): 
Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: 
Inserire il periodo che intercorre tra le rate in numero di mesi: 
Inserire la convenzione:
Inserire la regola per il calcolo delle date:


In [6]:
calendar = TARGET()
start_date = calendar.advance(ref_date,2,Days)
if maturity_label[-1] == 'M':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
elif maturity_label[-1] == 'Y':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
Tenor=Period(tenor,Months)
if (business_convention == 'ModifiedFollowing'):
  convention=1
elif (business_convention == 'ModifiedFollowing'):
    convention=0
terminationDateConvention= business_convention
if (date_gen_rule == 'Forward'):
  rule=1
elif (date_gen_rule == 'Backward'):
  rule=0
endofmonth = False
schedule = Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth)
lista_startdate = pd.DataFrame({'startdate':list(schedule)})
schedule_end = Schedule(start_date+Tenor,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth)
lista_enddate = pd.DataFrame({'enddate':list(schedule_end)})

**Ammortamento Francese**

In [7]:
#Calcolo della rata 
if maturity_label[-1] == 'M':
  rate_totali = int(12/tenor)
elif maturity_label[-1] == 'Y':
  rate_totali = int((12/tenor)*int(maturity_label.rsplit(maturity_label[-1], 1)[0]))
t = (1+ amortization_rate/(12/tenor))**rate_totali
rata=round(notional*t*(amortization_rate/(12/tenor))/(t-1),2)
Residuo=notional
tot_I=0
Estinto=0
plan=np.array
plan=[[0,0,0,0,0,Residuo]]

In [8]:
g=(1+1/((1+0.05/4)**rate_totali-1))*notional*0.0125


In [9]:
(amortization_rate/(12/tenor))
((1+amortization_rate)**(tenor/12)-1)
amortization_rate*90/360

0.0125

In [10]:
rate=range(0,rate_totali)
for count in rate:
  Int_quota=round(Residuo*(amortization_rate*(tenor/12)),2)
  Cap_quota=(rata-Int_quota)
  tot_I=(tot_I+Int_quota)
  Residuo=(Residuo-Cap_quota)
  Estinto=(Estinto+Cap_quota)
  Tot=(Int_quota+Cap_quota)
  plan.append([Int_quota,Cap_quota,Tot,tot_I,Estinto,Residuo])

In [11]:
index=['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
df=pd.DataFrame(plan)
df.columns=index
df1=pd.concat([lista_startdate,df],axis=1)
df2=pd.concat([lista_startdate,lista_enddate,df['Debito Residuo']],axis=1)
df2=df2[:-1]

**Ammortamento** **Italiano**

In [12]:
Residuo_ita=notional
tot_I_ita=0
Estinto_ita=0
Tot_ita=0
plan_ita=np.array
plan_ita=[[0,0,0,0,0,Residuo_ita]]

In [13]:
rate_ita=range(0,rate_totali)
for count in rate_ita:
  Int_quota_ita=round(Residuo_ita*amortization_rate/(12/tenor),2)
  Cap_quota_ita=round((notional/rate_totali),2)
  tot_I_ita=(tot_I_ita+Int_quota_ita)
  Residuo_ita=(Residuo_ita-Cap_quota_ita)
  Estinto_ita=(Estinto_ita+Cap_quota_ita)
  Tot_ita=(Int_quota_ita+Cap_quota_ita)
  plan_ita.append([Int_quota_ita,Cap_quota_ita,Tot_ita,tot_I_ita,Estinto_ita,Residuo_ita])

In [14]:
index=['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
df_ita=pd.DataFrame(plan_ita)
df_ita.columns=index
df1_ita=pd.concat([lista_startdate,df_ita],axis=1)
df2_ita=pd.concat([lista_startdate,lista_enddate,df_ita['Debito Residuo']],axis=1)
df2_ita=df2_ita[:-1]

**Ammortamento** **Bullet**

In [15]:
Residuo_bullet=notional
tot_I_bullet=0
plan_bullet=np.array
plan_bullet=[[0,0,Residuo_bullet]]
rate_bullet=range(0,rate_totali)
for count in rate_bullet:
  Int_quota_bullet=round(Residuo_bullet*amortization_rate/(12/tenor),2)
  tot_I_bullet=(tot_I_bullet+Int_quota_bullet)
  plan_bullet.append([Int_quota_bullet,tot_I_bullet,Residuo_bullet])

In [16]:
index=['Quota Interesse','Interessi Totali','Debito Residuo']
df_bullet=pd.DataFrame(plan_bullet)
df_bullet.columns=index
df1_bullet=pd.concat([lista_startdate,df_bullet],axis=1)
df2_bullet=pd.concat([lista_startdate,lista_enddate,df_bullet['Debito Residuo']],axis=1)
df2_bullet=df2_bullet[:-1]

**Output del Modello**

In [17]:
if (amortization_type== 'Francese'):
  print (df2)
elif (amortization_type == 'Italiano'):
  print (df2_ita)
elif (amortization_type == 'Bullet'):
  print (df2_bullet)

             startdate             enddate  Debito Residuo
0     April 26th, 2021     July 26th, 2021        100000.0
1      July 26th, 2021  October 26th, 2021        100000.0
2   October 26th, 2021  January 26th, 2022        100000.0
3   January 26th, 2022    April 26th, 2022        100000.0
4     April 26th, 2022     July 26th, 2022        100000.0
5      July 26th, 2022  October 26th, 2022        100000.0
6   October 26th, 2022  January 26th, 2023        100000.0
7   January 26th, 2023    April 26th, 2023        100000.0
8     April 26th, 2023     July 26th, 2023        100000.0
9      July 26th, 2023  October 26th, 2023        100000.0
10  October 26th, 2023  January 26th, 2024        100000.0
11  January 26th, 2024    April 26th, 2024        100000.0
12    April 26th, 2024     July 26th, 2024        100000.0
13     July 26th, 2024  October 28th, 2024        100000.0
14  October 28th, 2024  January 27th, 2025        100000.0
15  January 27th, 2025    April 28th, 2025        100000

In [18]:
market_data_path = '/content/drive/MyDrive/data.xlsx'
fixed_rates = pd.read_excel(market_data_path, sheet_name='fixings')
estimation_rate = pd.read_excel(market_data_path, sheet_name='estimation')
discount_rate = pd.read_excel(market_data_path, sheet_name='discount')

In [19]:
class IRS:

  def __init__(self, leg1_name = 'Receive', leg2_name = 'Pay', leg1_rate_type = 'Fixed', leg2_rate_type = 'Floating'):

    self.start_date = None
    self.leg1_flows = pd.DataFrame
    self.leg2_flows = pd.DataFrame
    self.leg1_name = leg1_name
    self.leg2_name = leg2_name
    self.leg1_rate_type = leg1_rate_type  
    self.leg2_rate_type = leg2_rate_type
    self.npv = None

In [20]:
def submit(path,
           underlying,
           amortization_type,
           notional,
           amortization_rate,
           maturity_label,
           ref_date,
           tenor,
           business_convention,
           date_gen_rule,
           currency = 'EUR',
           calendar = TARGET(),
           end_of_month = False,
           deduction_formula = '-1M LBD/-1BD',
           day_count = Actual360):

  irs = IRS()

In [21]:
submit(path = market_data_path, underlying = underlying, amortization_type = amortization_type, notional = notional, amortization_rate = amortization_rate, maturity_label = maturity_label, ref_date = ref_date, tenor = tenor,
       business_convention = business_convention, date_gen_rule = date_gen_rule)

In [22]:
leg_fixed=FixedRateLeg(schedule,Actual360(),[notional],[0.05])
for i in leg_fixed:
  print(i.date(),i.amount())

July 26th, 2021 1263.8888888888866
October 26th, 2021 1277.7777777777776
January 26th, 2022 1277.7777777777776
April 26th, 2022 1249.9999999999955
July 26th, 2022 1263.8888888888866
October 26th, 2022 1277.7777777777776
January 26th, 2023 1277.7777777777776
April 26th, 2023 1249.9999999999955
July 26th, 2023 1263.8888888888866
October 26th, 2023 1277.7777777777776
January 26th, 2024 1277.7777777777776
April 26th, 2024 1263.8888888888866
July 26th, 2024 1263.8888888888866
October 28th, 2024 1305.5555555555597
January 27th, 2025 1263.8888888888866
April 28th, 2025 1263.8888888888866
July 28th, 2025 1263.8888888888866
October 27th, 2025 1263.8888888888866
January 26th, 2026 1263.8888888888866
April 27th, 2026 1263.8888888888866


prove npv obiettivo


In [27]:
df2['bbb']=df2['Debito Residuo']*df2['aaa']
df2

,startdate,enddate,Debito Residuo,aaa,bbb
0,"April 26th, 2021","July 26th, 2021",100000.00,91,9.1e+06
1,"July 26th, 2021","October 26th, 2021",95567.96,92,8.79225e+06
2,"October 26th, 2021","January 26th, 2022",91080.52,92,8.37941e+06
3,"January 26th, 2022","April 26th, 2022",86536.99,90,7.78833e+06
4,"April 26th, 2022","July 26th, 2022",81936.66,91,7.45624e+06
5,"July 26th, 2022","October 26th, 2022",77278.83,92,7.10965e+06
6,"October 26th, 2022","January 26th, 2023",72562.78,92,6.67578e+06
7,"January 26th, 2023","April 26th, 2023",67787.77,90,6.1009e+06
8,"April 26th, 2023","July 26th, 2023",62953.08,91,5.72873e+06
9,"July 26th, 2023","October 26th, 2023",58057.95,92,5.34133e+06


In [26]:
df2['aaa']

0     91
1     92
2     92
3     90
4     91
5     92
6     92
7     90
8     91
9     92
10    92
11    91
12    91
13    94
14    91
15    91
16    91
17    91
18    91
19    91
Name: aaa, dtype: object